In [2]:
import gc
import pandas as pd
import numpy as np
import lightgbm as lgb
from scipy.stats import pearsonr
import joblib
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split, GroupKFold
import warnings
warnings.simplefilter('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [3]:
# data = pd.read_csv("C:\\Users\\Mengxiao.Wu\\Desktop\\train\\train.csv")
# data.to_hdf("C:\\Users\\Mengxiao.Wu\\Desktop\\train\\train.h5", "train")

In [4]:
# data.to_hdf("C:\\Users\\Mengxiao.Wu\\Desktop\\train\\train.h5", "train")

In [5]:
%%time
data = pd.read_hdf("C:\\Users\\Mengxiao.Wu\\Desktop\\train\\winsorize.h5", "train")
print(data.shape)

(3140461, 304)
Wall time: 45.7 s


In [6]:
data['investment_id'] = data['investment_id'].astype('category')
feats = [col for col in data.columns if col not in ['time_id', 'row_id', 'target']]

In [7]:
# # 参数
# params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',  # 设置提升类型
#     'objective': 'regression',  # 目标函数
#     'metric': {'auc'},  # 评估函数
#     'max_bin': 255,  # 大会有更准的效果,更慢的速度
#     'min_data_in_leaf': 91,
#     'num_iterations': 500,
#     'max_depth': 4,  # 指定树的最大深度 [3, 5, 6, 7, 9, 12, 15, 17, 25]
#     'num_leaves': 10,  # 叶子节点数
#     'learning_rate': 0.1,  # 学习速率 [0.01, 0.015, 0.025, 0.05, 0.1]
#     'feature_fraction': 1.0,  # 建树的特征选择比例 [0.6, 0.7, 0.8, 0.9, 1]
#     'bagging_fraction': 0.6,  # 建树的样本采样比例 [0.6, 0.7, 0.8, 0.9, 1]
#     # 'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
#     'min_sum_hessian_in_leaf': 3.0,  # 防止过拟合
#     'verbose': 1  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
# }
params = {
        'learning_rate':0.1,
        "objective": "regression",
        "metric": "rmse",
        'boosting_type': "gbdt",
        'verbosity': -1,
        'n_jobs': -1, 
        'seed': 21,
        'lambda_l1': 1.1895057699067542, 
        'lambda_l2': 1.9079686837880768e-08, 
        'num_leaves': 112, 
        'subsample':None,
        'feature_fraction': 0.6259927292757151, 
        'bagging_fraction': 0.9782210574588895, 
        'bagging_freq': 1, 
        'n_estimators': 306, 
        'max_depth': 12, 
        'max_bin': 255, 
        'min_data_in_leaf': 366,
        'colsample_bytree': None,
        'subsample_freq': None,
        'min_child_samples': None,
        'reg_lambda': None,
        'reg_alpha': None,
    }

In [8]:
n_splits = 10
kfold = GroupKFold(n_splits)
fold_scores = []
models = []


In [9]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kfold.split(data[feats], data.target, groups=data.time_id)):
    X_train, y_train = data[feats].iloc[trn_idx], data['target'].iloc[trn_idx]
    X_val, y_val = data[feats].iloc[val_idx], data['target'].iloc[val_idx]

    model = lgb.LGBMRegressor(**params)

    model.fit(X_train, y_train, eval_metric='rmse', eval_set=[(X_val, y_val)], verbose=100, early_stopping_rounds=50)

    joblib.dump(model, f'lgbm_fold_{fold}.pkl')
    md = joblib.load(f'lgbm_fold_{fold}.pkl')
    y_pred = md.predict(X_val)
#     y_pred = model.predict(X_val)
#     y_pred = []
#     for model in models:
#         y_pred.append(model.predict(X_val))

#     y_pred = np.nanmean(y_pred, axis = 0)

    score = pearsonr(y_pred, y_val)[0]
    print(f"Fold {fold + 1}: {score}")

    fold_scores.append(score)
    models.append(model)

    del model, y_pred, score, X_train, y_train, X_val, y_val
    gc.collect()
    
print(f"Overall score: {np.mean(fold_scores, axis=0)}")

[100]	valid_0's rmse: 0.912714
Fold 1: 0.13155193404420915
[100]	valid_0's rmse: 0.909715
[200]	valid_0's rmse: 0.909467
Fold 2: 0.13787007905067414
[100]	valid_0's rmse: 0.906641
[200]	valid_0's rmse: 0.906251
Fold 3: 0.1503298697450862
[100]	valid_0's rmse: 0.915343
[200]	valid_0's rmse: 0.914865
[300]	valid_0's rmse: 0.914713
Fold 4: 0.14380118086217256
[100]	valid_0's rmse: 0.908924
[200]	valid_0's rmse: 0.908308
[300]	valid_0's rmse: 0.908199
Fold 5: 0.15249867329671551
[100]	valid_0's rmse: 0.909472
[200]	valid_0's rmse: 0.908865
[300]	valid_0's rmse: 0.908688
Fold 6: 0.14902497024662895
[100]	valid_0's rmse: 0.908329
[200]	valid_0's rmse: 0.907874
Fold 7: 0.1507330766044804
[100]	valid_0's rmse: 0.90758
[200]	valid_0's rmse: 0.907123
Fold 8: 0.15570067679368396
[100]	valid_0's rmse: 0.910214
Fold 9: 0.12809671929567543
[100]	valid_0's rmse: 0.914056
[200]	valid_0's rmse: 0.913209
[300]	valid_0's rmse: 0.912871
Fold 10: 0.16068707995530573
Overall score: 0.1460294259894632
Wall t

In [10]:
pkl = []
for i in range(10):
    model_name = f'lgbm_fold_{i}.pkl'
    with open(model_name, 'rb') as f:
        p = joblib.load(f)
        print(p)
    pkl.append(p)

LGBMRegressor(bagging_fraction=0.9782210574588895, bagging_freq=1,
              colsample_bytree=None, feature_fraction=0.6259927292757151,
              lambda_l1=1.1895057699067542, lambda_l2=1.9079686837880768e-08,
              max_bin=255, max_depth=12, metric='rmse', min_child_samples=None,
              min_data_in_leaf=366, n_estimators=306, num_leaves=112,
              objective='regression', reg_alpha=None, reg_lambda=None, seed=21,
              subsample=None, subsample_freq=None, verbosity=-1)
LGBMRegressor(bagging_fraction=0.9782210574588895, bagging_freq=1,
              colsample_bytree=None, feature_fraction=0.6259927292757151,
              lambda_l1=1.1895057699067542, lambda_l2=1.9079686837880768e-08,
              max_bin=255, max_depth=12, metric='rmse', min_child_samples=None,
              min_data_in_leaf=366, n_estimators=306, num_leaves=112,
              objective='regression', reg_alpha=None, reg_lambda=None, seed=21,
              subsample=None, subsamp

In [11]:
scores = []
for fold, (trn_idx, val_idx) in enumerate(kfold.split(data[feats], data.target, groups=data.time_id)):
    X_train, y_train = data[feats].iloc[trn_idx], data['target'].iloc[trn_idx]
    X_val, y_val = data[feats].iloc[val_idx], data['target'].iloc[val_idx]
    
    y_pred = []
    for p in pkl:
        y_pred.append(p.predict(X_val))

    y_pred = np.mean(y_pred, axis = 0)

    score = pearsonr(y_pred, y_val)[0]
    print(f"Fold {fold + 1}: {score}")

    scores.append(score)

    del y_pred, score, X_train, y_train, X_val, y_val
    gc.collect()
    
print(f"Overall score: {np.mean(scores, axis=0)}")

Fold 1: 0.4180587634659933
Fold 2: 0.40606839772575015
Fold 3: 0.42077108826240767
Fold 4: 0.40933076411135055
Fold 5: 0.42032121217273893
Fold 6: 0.41401735029141
Fold 7: 0.42719775995834913
Fold 8: 0.4201127575169891
Fold 9: 0.4214549390588938
Fold 10: 0.42678793270824766
Overall score: 0.41841209652721306


In [18]:
scores = []
print(1)
X, y = data[feats], data['target']
y_pred = []
for model in models:
    y_pred.append(model.predict(X))
    
print(len(y_pred))

y_pred = np.mean(y_pred, axis = 0)

score = pearsonr(y_pred, y)[0]
print(f"Overall score: {score}")

1
10
Overall score: 0.4184383143456026


In [13]:
tmp = pd.read_csv("C:\\Users\\Mengxiao.Wu\\Desktop\\train\\train.csv")
X = tmp.drop(['row_id', 'time_id', 'investment_id', 'target'], axis=1)

X = np.mean(X, axis=0)
print(X)

f_0      0.001637
f_1     -0.012578
f_2      0.008515
f_3     -0.020037
f_4     -0.005416
           ...   
f_295   -0.027737
f_296   -0.001518
f_297   -0.007058
f_298   -0.004217
f_299   -0.004319
Length: 300, dtype: float64
